Day 7: Clean solution

* Create a Tree structure
* Traverse

In [ ]:
class Folder:
    def __init__(self, name, parent=None):
        self.name = name
        self.parent = parent
        self.subfolders = []
        self.files = []

    def get_size(self):
        size = 0
        for file in self.files:
            size += file.size
        for subfolder in self.subfolders:
            size += subfolder.get_size()
        return size

    def add_subfolder(self, subfolder):
        self.subfolders.append(subfolder)
        subfolder.parent = self

    def add_file(self, file):
        self.files.append(file)
        
    def find_subfolder(self, name):
        for subfolder in self.subfolders:
            if subfolder.name == name:
                return subfolder
        return None
    
    def _print_folder(self, indent):
        result = indent + self.name + "\n"
        for file in self.files:
            result += indent + "  " + file.name + " (file, size=" + str(file.size) + ")" + "\n"
        for subfolder in self.subfolders:
            result += subfolder._print_folder(indent + "  ")
        return result

class File:
    def __init__(self, name, size, parent=None):
        self.name = name
        self.size = size
        self.parent = parent



inputfile = open("./input7.txt", "r").read()


def buildFolder(input):
    root = Folder('root')
    currentfolder = root
    for line in input.split('\n'):
        
        if line.startswith("$ cd"):
            # go to parent
            if line == "$ cd ..":
                currentfolder = currentfolder.parent
            elif line == "$ cd /": 
                currentfolder = root
            else:
                currentfolder = currentfolder.find_subfolder(line.replace("$ cd ", ""))
        
        elif line.startswith("dir"):
            newfoldername = line.replace("dir ", "")
            currentfolder.add_subfolder(Folder(newfoldername))
        
        elif line.startswith("$") == False and len(line) > 0:
            splitline = line.split(" ")
            newfile = File(splitline[1], int(splitline[0]))
            currentfolder.add_file(newfile)
    
    return root

rootfolder = buildFolder(inputfile)
# print(rootfolder._print_folder(" "))


def traverse(folder):
    
    total_sizes = []
    curr_total = 0 
    for file in folder.files:
        curr_total += file.size
    
    for subfolder in folder.subfolders:
        total_sizes.extend(traverse(subfolder))
        curr_total += total_sizes[-1]
    
    total_sizes.append(curr_total)
    return total_sizes

foldersizes = traverse(rootfolder)


#part one
def getSumSmallFolders(folderlist):
    sum = 0
    for item in folderlist:
        if item <= 100000:
            sum += item
    return sum


print(getSumSmallFolders(foldersizes))



#part two
def findToDeleteFolder(folderlist):
    max = 70000000
    needed = 30000000
    sortedlist = sorted(folderlist)
    current = sortedlist[-1]
    todelete = current - (max - needed)
    
    for folder in sortedlist:
        if folder >= todelete:
            return folder
    
    
print(findToDeleteFolder(foldersizes))    

            

Day 7: Hacky solution (part one)

I've tried doing a hacky solution first, putting all the folders in a list. And then using recursion to go through the list

In [10]:
input = open("./input7.txt", "r").read()

def getAllDirs(testinput):
    alldir = []
    currentpath = []
    for index, line in enumerate(testinput.split('\n')):
        currentDir = "/".join(currentpath)
        
        printHelp = False
    
        if printHelp: print(line)
        if printHelp: print(index)
        if printHelp: print("current dir = " + currentDir)
        #moving 
        if line.startswith("$ cd"):
            if line == "$ cd ..":
                if printHelp : print("move up")
                currentpath.pop()
                
            else:
                if printHelp : print("move to " + line.replace("$ cd ", ""))
                currentpath.append(line.replace("$ cd ", ""))
                
        if line.startswith("dir"):
            # create a new dir
            dirname = currentDir + "/" + line.replace("dir ", "")
            newDir = {
                "name": dirname,
                "filesize": 0,
                "children": []
            }
            alldir.append(newDir)
            
            # add dir to list of children
            for dir in alldir:
                if dir["name"] == currentDir:
                    if printHelp: print("add " +  dirname + " to " + dir["name"])
                    dir["children"].append(dirname)

        if line.startswith("$") != True and line.startswith("dir") != True and len(line) > 0:
            if printHelp: print("adding file")
            for dir in alldir:
                if dir["name"] == currentDir:
                    dir["filesize"] += int(line.split(" ", 1)[0])
                
    return alldir        


# calculate childrenSize
def checkChildrenSize(dir, alldirs):
    if len(dir["children"]) == 0:
        return
    else:
        childrenSize = 0
        for child in dir["children"]:     
            for alldiritem in alldirs:
                if alldiritem["name"] == child:
                    checkChildrenSize(alldiritem, alldirs)
                    childrenSize += alldiritem["filesize"]
                    
        for alldiritem in alldirs:
            if dir["name"] == alldiritem["name"]:
                alldiritem["filesize"] += childrenSize
                
alldir = getAllDirs(input)
print(len(alldir))

sumdirs = 0
for dir in alldir:
    # check if it's a child
    if "/" not in dir["name"][2:]:
        checkChildrenSize(dir, alldir)
    if int(dir["filesize"]) <= 100000:
        sumdirs += int(dir["filesize"])

print(sumdirs)

180
1543140
